In [2]:
from PIL import Image
import numpy
from sklearn.cluster import MiniBatchKMeans
import multiprocessing as mp
from functools import partial
from itertools import repeat
import os
import json
import time

'''
Quelques notes :

k-Means : 
 * cet algorithme permet de rechercher un nombre prédéterminé (NB_CLUSTER)
de grappes (appelé cluster) dans une image (dans notre application). 
 * le centre de chaque grappe est la moyenne arithmétique de tous les points appartenant à la grappe
'''
## Temps d'éxécutions
# KMeans environ  : 160s pour 70 fichiers
# MiniBatchKMeans : 70s pour 70 fichiers

# Settings dataset folders
POKE_DATA_DIRECTORY = '../datasets/pokedata'
POKE_DATA_DIRECTORY_TEST = POKE_DATA_DIRECTORY+'/Muk'

# Nombre de valeur K-MEANS
NB_CLUSTER = 3


#########################################################################################################
# 
# Cette classe permet de récupérer le chemin (os) des fichiers d'un dossier et/ou de ses sous répertoires
#
class LocalDataset:
    subDirectories = [] # Contient le chemin de sous dossier
    files = []
    dictionnary = {}
    countFiles = 0
    
    def __init__(self,rootDirectory,subDirectory=None):
        self.rootDirectory = rootDirectory
        if subDirectory == None:
            self.setPathSubDirectory()
        else:
            self.subDirectories.append(subDirectory)
        self.setDictFolderFiles()
        
    def setPathSubDirectory(self):
        for directory in os.listdir(self.rootDirectory):
            self.subDirectories.append(self.rootDirectory+'/'+directory)
            
    def setDictFolderFiles(self):
        for subRep in self.subDirectories:
            name = subRep.split('/')
            name = name[-1]
            self.dictionnary[name] = []
            for file in os.listdir(subRep):
                if (not(".svg" in file)):
                    self.dictionnary[name].append(subRep+"/"+file)
                    self.files.append(subRep+'/'+file)
                    self.countFiles +=1
    
    def getDictFolderFiles(self):
        return self.dictionnary

    def getSubDirectories(self):
        return self.subDirectories
    
    def getCountFiles(self):
        return self.countFiles
#
#
#########################################################################################################



#########################################################################################################
# 
# Cette classe d'enregistrer un dictionnaire dans un fichier json
#
class JSONWriter:
    def __init__(self, filePath):
        self.filePath = filePath

    def writeJSON(self, data):
        with open(self.filePath, 'w') as f:
            json.dump(data, f)
#
#
#########################################################################################################
 

           
#########################################################################################################
# 
# Cette classe permet gérer la création de nos images
#
class ImageManager:
    def __init__(self):
        self.images = []
    
    # Création d'une image et sauvegarde de sa référence
    def addImage(self, name, imgPath):
        self.images.append(ImageP(name,imgPath))
          
    def getImages(self):
        return self.images
#
#
#########################################################################################################   



#########################################################################################################
# 
# Cette classe représente nos images
#
class ImageP:
    
    
    def __init__(self, name, path):
        self.name = name # nom du pokemon -> nom du dossier
        self.path = path # chemin de l'image
        self.clusterCenters = {} # centre des NB_CLUSTER grappes
        self.format = None 
        self.size = None # size -> (width,height)

    
    def setKmeans(self,cluster_centers):
        self.clusterCenters["colors"]=[]
        for color in cluster_centers:
            self.clusterCenters["colors"].append({"red":color[0], "green":color[1], "blue":color[2]})
    
    def setFormat(self, imageFormat):
        self.format = imageFormat
    
    def setSize(self, size):
        self.size = size
    
    def toDict(self):
        imageDict = {}
        imageDict["name"]= self.name
        imageDict["path"] = self.path
        imageDict["colors"] = []
        
        for color in self.clusterCenters:
            imageDict["colors"].append({"red":color[0], "green":color[1], "blue":color[2]})

        imageDict["size"] = self.size
        imageDict["format"] = self.format
        
        return imageDict
        
        
        
#
#
#########################################################################################################



# Cette fonction calcule les k-means valeurs pour NB_CLUSTER grappes
def processingKmeans(image,return_list):
    print("-> Processing image : "+image.path)
    
    im = Image.open(image.path) # Création objet PIL
    
    image.setFormat(im.format) # Définit le format de l'objet image
    image.setSize(im.size) # Définit la taille de l'objet image
    
    
    numarray = numpy.array(im.getdata(), numpy.uint8) # Création array pixel rgb
    dim = numarray.ndim # Dimension de l'array
    
    if dim == 2: # clusters.fit n'accepte pas les images à 1 dimension
        clusters = MiniBatchKMeans(n_clusters = NB_CLUSTER)
        clusters.fit(numarray)
        image.setKmeans(clusters.cluster_centers_)
        
        return_list.append(image)
        print("Done process :"+image.path)
        

        
# Lancement du chronomètre
startTime = time.time()


# Initialisation des données
dataset = LocalDataset(POKE_DATA_DIRECTORY,POKE_DATA_DIRECTORY_TEST)
allFiles = dataset.files


# Initialisation des objets images
imageManager = ImageManager()
images = []

for file in allFiles:
    pokemonName=file.split('/')
    imageManager.addImage(pokemonName[-1],file)

images = imageManager.getImages()
print("Nombre d'images à traiter : ",len(images))

    
# Initialisation du multiprocessing
processingManager = mp.Manager()
return_list_image = processingManager.list()
pool = mp.Pool(processes=4)


# Création / Lancement des processus
pool.starmap(processingKmeans, zip(images, repeat(return_list_image)))


# Préparation des données à exporter
exportData = []
for image in return_list_image:
    exportData.append(image.toDict())

# Export des données en JSON
jw = JSONWriter("./datasets/data.json") # Fichier de sortie

print("Running -> Object to JSON ...")
jw.writeJSON(exportData)
print("Done -> Object to JSON ...")


# Arrêt du chronomètre
endTime = time.time()
totalTime = endTime - startTime
print("Total time: ", totalTime)


Nombre d'images à traiter :  71
-> Processing image : ../datasets/pokedata/Muk/d97fccb731a84cdc9c3872e9b0396fbe.jpg
-> Processing image : ../datasets/pokedata/Muk/620187e98117499694d16d11c6992264.jpg
-> Processing image : ../datasets/pokedata/Muk/f393f240104646849ef3121a0b13660d.jpg
-> Processing image : ../datasets/pokedata/Muk/726d39a4959a4bf3b0b81105bf739b56.jpg
Done process :../datasets/pokedata/Muk/726d39a4959a4bf3b0b81105bf739b56.jpg
-> Processing image : ../datasets/pokedata/Muk/7b60685b53454ffc9647d9b4fb073249.jpg
Done process :../datasets/pokedata/Muk/f393f240104646849ef3121a0b13660d.jpg
-> Processing image : ../datasets/pokedata/Muk/f70664da7b154e469a52e6bcbb7ddac0.jpg
Done process :../datasets/pokedata/Muk/f70664da7b154e469a52e6bcbb7ddac0.jpg
-> Processing image : ../datasets/pokedata/Muk/db7686074ea14fc18f4a8e7ad1b1aada.jpg
Done process :../datasets/pokedata/Muk/db7686074ea14fc18f4a8e7ad1b1aada.jpg
-> Processing image : ../datasets/pokedata/Muk/020b12c6be0744d8ac3e3b4cbdd1f

Done process :../datasets/pokedata/Muk/97161eb635bd49fe86329e3064b11c0f.jpg
-> Processing image : ../datasets/pokedata/Muk/ecdf8c79e57947f092c230e298ca92b8.jpg
Done process :../datasets/pokedata/Muk/e03de401122c42ce831160a59b0678df-3.jpg
-> Processing image : ../datasets/pokedata/Muk/1b55a4ec96f14690b1d9febdd4540f86.jpg
Done process :../datasets/pokedata/Muk/1b55a4ec96f14690b1d9febdd4540f86.jpg
-> Processing image : ../datasets/pokedata/Muk/6551686bda8f4c4081a63a3bac1f3ea3.jpg
Done process :../datasets/pokedata/Muk/be0f8412d6c448ce94c41f5d2bd9abb3.jpg
-> Processing image : ../datasets/pokedata/Muk/4d6af95f821440b985af52a3e38a03c1.jpg
Done process :../datasets/pokedata/Muk/4d6af95f821440b985af52a3e38a03c1.jpg
-> Processing image : ../datasets/pokedata/Muk/a21e7b5cbe8b4a88adefdd683623c0dc.jpg
Done process :../datasets/pokedata/Muk/6551686bda8f4c4081a63a3bac1f3ea3.jpg
-> Processing image : ../datasets/pokedata/Muk/472a38fc09e34d1bace624fc8d002355.jpg
Done process :../datasets/pokedata/Muk